In [1]:
import os
import random
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import Dataset, load_dataset
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

/workspaces/js-complexity-model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

import utils.evaluator as evaluator
evaluate = evaluator.evaluate

In [3]:
load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

dataset = load_dataset("aslam-naseer/js-function-complexity-processed")

Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


### Neural Network

In [4]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [5]:
train_ds = dataset['train']
val_ds = dataset['validation']
test_ds = dataset['test']

feature_cols = [
    'param_count',
    'local_statement_count',
    'total_statement_count',
    'local_variable_count',
    'total_variable_count',
    'local_nesting_depth',
    'total_nesting_depth'
]

train_array = np.array([train_ds[col] for col in feature_cols]).T
scaler = StandardScaler()
scaler.fit(train_array)

,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True


In [6]:
def normalise_features(data):
  mat = np.array([data[col] for col in feature_cols]).reshape(1,-1)
  scaled_mat = scaler.transform(mat)
  return {"features": scaled_mat}

train_ds = train_ds.map(normalise_features, num_proc=4)
val_ds = val_ds.map(normalise_features, num_proc=4)

train_ds.set_format(type='torch', columns=['features', 'complexity'])
val_ds.set_format(type='torch', columns=['features', 'complexity'])

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2) 

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [21]:
def train_model(num_epochs, model, train_loader, val_ds, criterion, optimizer):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch in train_loader:
            inputs = batch['features'].float()
            labels = batch['complexity'].float().unsqueeze(1)

            if inputs.dim() == 3:
                inputs = inputs.squeeze(1)

            optimizer.zero_grad()
            outpus = model(inputs)
            loss = criterion(outpus, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            val_loader = DataLoader(val_ds, batch_size=32)
        for batch in val_loader:
            inputs = batch['features'].float()
            labels = batch['complexity'].float().unsqueeze(1)

            if inputs.dim() == 3:
                inputs = inputs.squeeze(1)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

        if (epoch + 1) % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], '
                f'Train Loss: {train_loss/len(train_loader):.4f}, '
                f'Val Loss: {val_loss/len(val_loader):.4f}')

In [22]:
num_epochs = 250
set_seed(42)

input_size = len(feature_cols)
model = NeuralNetwork(input_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

train_model(num_epochs, model, train_loader, val_ds, criterion, optimizer)

Epoch [50/250], Train Loss: 1.3653, Val Loss: 1.1810
Epoch [100/250], Train Loss: 1.2008, Val Loss: 1.0667
Epoch [150/250], Train Loss: 1.0618, Val Loss: 0.9403
Epoch [200/250], Train Loss: 0.9867, Val Loss: 0.9264
Epoch [250/250], Train Loss: 0.9620, Val Loss: 0.9047


In [23]:
def nn_predict(data):
  model.eval()
  with torch.no_grad():
    input = normalise_features(data)['features']
    output = model(torch.tensor(input).float())
    return output[0].item()
  
evaluate(nn_predict, test_ds)

  0%|          | 0/500 [00:00<?, ?it/s]

 38%|███▊      | 189/500 [00:00<00:00, 1881.95it/s]

0.2 1.8 0.4 1.2 0.4 0.0 0.2 0.9 0.7 0.8 0.6 0.7 1.6 0.3 0.3 3.5 2.1 0.0 0.4 0.3 1.0 1.3 0.2 1.0 0.1 0.1 1.9 0.5 0.5 0.3 0.1 2.0 0.3 0.0 0.7 0.4 0.3 0.4 0.9 0.1 1.0 0.4 0.9 1.2 1.2 0.5 0.4 0.3 0.3 2.1 0.2 0.0 0.9 0.5 0.3 0.2 1.2 1.1 0.6 0.2 0.3 0.0 0.5 0.1 0.4 0.1 0.2 0.1 0.3 0.2 0.6 0.3 1.3 3.6 0.9 0.7 0.3 0.4 0.4 0.5 0.7 0.2 1.0 0.0 0.0 0.6 0.8 0.7 2.9 0.1 1.0 0.8 1.0 1.0 0.7 0.4 0.3 0.8 0.2 2.2 0.3 0.5 0.0 0.8 0.4 1.1 2.0 2.0 0.2 0.9 0.3 0.5 0.7 0.2 0.5 0.0 1.6 2.4 0.3 0.8 0.0 0.3 0.3 1.4 0.7 0.8 0.2 0.2 0.2 0.4 0.7 0.0 0.1 0.7 1.6 0.1 1.0 0.1 0.2 0.2 0.2 0.3 0.6 0.7 0.3 0.2 0.1 0.3 0.2 0.5 0.0 0.4 0.8 0.7 0.9 0.6 0.4 0.6 0.4 0.0 0.6 0.0 1.0 1.0 0.3 0.8 0.4 0.2 0.6 0.3 0.2 0.5 0.2 0.2 0.4 0.1 0.0 0.4 0.4 0.6 0.4 0.3 0.2 0.0 0.5 0.3 0.5 0.1 0.4 0.2 0.1 0.1 0.4 0.5 3.3 2.2 0.7 0.1 0.2 0.1 0.8 0.6 1.3 0.6 0.3 0.2 0.4 1.3 0.8 2.9 0.0 0.6 0.4 0.6 0.0 0.7 0.6 0.2 0.4 1.5 0.2 0.4 0.9 2.2 0.7 0.8 0.5 0.4 1.0 0.4 0.1 0.4 0.5 1.5 0.4 0.4 0.6 0.1 1.4 0.2 0.6 0.6 0.2 0.9 0.9 0.3 0.1 0.3 0.5 0.3 

100%|██████████| 500/500 [00:00<00:00, 1831.59it/s]

0.2 0.1 0.1 0.1 0.4 0.1 0.6 0.1 0.4 2.0 0.0 0.1 0.5 0.3 0.3 1.2 1.1 0.3 0.2 1.5 1.5 1.3 0.5 0.1 1.4 0.8 3.6 1.3 0.2 1.6 0.5 1.0 0.1 0.9 0.1 0.2 0.2 0.2 0.8 0.4 1.5 0.8 0.8 0.2 1.1 0.1 0.1 0.9 0.5 0.5 1.0 0.4 1.3 0.7 1.0 1.3 0.2 0.4 0.2 1.2 0.4 0.9 0.6 0.0 0.3 0.6 0.6 0.8 0.9 0.6 0.5 0.3 1.0 0.1 0.9 0.4 0.2 1.0 0.5 0.1 0.2 0.7 0.1 0.1 0.6 0.5 2.7 0.1 1.7 0.3 1.0 0.1 2.3 1.9 0.7 0.2 0.2 0.6 0.3 1.0 0.2 0.8 2.3 0.9 0.2 0.2 0.3 0.7 0.4 0.7 1.4 0.2 0.8 0.1 1.7 0.6 1.4 0.6 0.1 4.4 0.1 1.3 0.4 